# ESTÁTICA

## Try 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Función para calcular el mapa de saliencia basado en el espectro de fase de la transformada de Fourier
def PFT_Color_Multiescala(img):
    # Si es una imagen en escala de grises, no podemos obtener información de los colores 
    if len(img.shape) == 2:
        print("Imagen en escala de grises, devolviendo la imagen original...")
        return img

    rows, cols = img.shape[0], img.shape[1]


    # Pasamos de BGR a LAB
    imageLAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Obtenemos los canales L->blanco-negro (luminosidad), A->rojo-verde, B->azul-amarillo
    L = imageLAB[:,:,0]
    A = imageLAB[:,:,1]
    B = imageLAB[:,:,2]
    channels = [L, A, B]
    results = []

    # Para cada canal
    for i in range(len(channels)):
        # Transformada de Fourier
        img_fft = np.fft.fft2(channels[i])

        # Espectro de la fase de la imagen
        phase_espectrum = np.angle(img_fft)

        # Mapa de saliencia
        sm = cv2.GaussianBlur((np.abs(np.fft.ifft2(np.exp(1j * phase_espectrum)))**2), (3, 3), 1)

        # Redimensionamos y normalizamos el mapa de saliencia
        sm = cv2.normalize(cv2.resize(sm, (rows, cols), cv2.INTER_LANCZOS4), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # Calcular la varianza del mapa de saliencia del canal actual
        var = np.var(sm)

        # Ponderar el mapa de saliencia del canal actual en función de la varianza
        results.append(sm / (var + 1e-8))   # Se agrega un pequeño valor para evitar la división por cero

    # Normalizar las ponderaciones para que sumen 1
    weights = np.array(results) / np.sum(results, axis=0)

    # Sumar los mapas ponderados
    combined_result = np.sum(weights * np.array(results), axis=0)

    return combined_result


# Cargar video
video_path = "./videos/SCE_bur_1_distractores_altos.avi"
cap = cv2.VideoCapture(video_path)

# Obtener información del video (dimensiones y fps)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Iterar sobre los fotogramas del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Llamamos a la función para calcular el mapa de saliencia a multiescala con información de color e invariante a la escala
    sm = PFT_Color_Multiescala(frame)

    # Visualización en tiempo real (opcional)
    cv2.imshow('Salient Region', sm)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar objetos y cerrar ventanas
cap.release()
cv2.destroyAllWindows()

## Try 2

In [ ]:
def PFT_generalizado(frame):
    ############ CONTINUAR ##################
    rows, cols = frame.shape[0], frame.shape[1]

    # Transformada de Fourier
    img_fft = np.fft.fft2(frame)

    # Espectro de la fase de la imagen
    phase_espectrum = np.angle(img_fft)

    # Mapa de saliencia
    sm = cv2.GaussianBlur((np.abs(np.fft.ifft2(np.exp(1j * phase_espectrum)))**2), (3, 3), 1)

    # Redimensionamos y normalizamos el mapa de saliencia
    sm = cv2.normalize(cv2.resize(sm, (rows, cols), cv2.INTER_LANCZOS4), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return sm

def prueba(prev_frame, current_frame):
    b0, g0, r0 = cv2.split(prev_frame)
    b, g, r = cv2.split(current_frame)

    # Colores
    R = r - (g + b)/2
    G = g - (r + b)/2
    B = b - (r + g)/2
    Y = (r + g)/2 - np.abs(r - g)/2 - b

    # Canales componente-opuesta de color
    RG = R - G
    BY = B - Y

    # Canal de intensidad
    I = (r + g + b)/3
    I_tau = (r0 + g0 + b0)/3

    # Canal de movimiento
    M = np.abs(I - I_tau)   # Tau = 1

    # Devolvemos los 4 canales: 2 de colores, 1 de intensidad y 1 de movimiento
    return RG, BY, I, M


# Función para calcular la saliencia basada en color utilizando el modelo Itti-Koch y luego invertir los colores
def color_SM(frame):
    # Convertir el fotograma a espacio de color LAB
    lab_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)

    # Separar los canales LAB
    l, a, b = cv2.split(lab_frame)

    # Normalizar los canales a valores entre 0 y 1
    l_normalized = cv2.normalize(l, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    a_normalized = cv2.normalize(a, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    b_normalized = cv2.normalize(b, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Calcular la saliencia basada en todos los canales LAB
    sm = cv2.GaussianBlur(l_normalized, (0, 0), sigmaX=2, sigmaY=2)
    sm += cv2.GaussianBlur(a_normalized, (0, 0), sigmaX=2, sigmaY=2)
    sm += cv2.GaussianBlur(b_normalized, (0, 0), sigmaX=2, sigmaY=2)

    # Escalar a valores entre 0 y 255 y luego invertir los colores
    sm = ((255 - (sm * 255 / np.max(sm))).astype(np.uint8))

    # Normalizamos los valores
    sm_norm = cv2.normalize(sm,None,0.0,255,cv2.NORM_MINMAX, cv2.CV_32F)

    # Mapa de calor
    sm_heated = cv2.applyColorMap(np.uint8(sm_norm), cv2.COLORMAP_JET)

    return sm_heated

# Función para calcular la saliencia basada en movimiento
def motion_SM(prev_frame, current_frame):
    # Convertir los fotogramas a escala de grises
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # Calcular el flujo óptico usando el método Farneback
    flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Calcular la magnitud del flujo óptico
    magnitude = np.sqrt(flow[:, :, 0] ** 2 + flow[:, :, 1] ** 2)

    # Normalizar la saliencia de movimiento
    sm = magnitude / np.max(magnitude)

    # Normalizamos los valores
    sm_norm = cv2.normalize(sm,None,0.0,255,cv2.NORM_MINMAX, cv2.CV_32F)

    # Mapa de calor
    sm_heated = cv2.applyColorMap(np.uint8(sm_norm), cv2.COLORMAP_JET)

    return sm_heated

# Función para combinar el mapa de saliencia con información de color y con movimiento
def GPhase_color_SM(color_sm, motion_sm):
    return (color_sm + motion_sm) / 2.0



# Leer el video
video = cv2.VideoCapture('./videos/SCE_bur_1_distractores_altos.avi')

# Leer el primer fotograma
ret, prev_frame = video.read()
#display_handle = display(None, display_id=True)
while True:
    ret, current_frame = video.read()
    if not ret:
        break

    # Calcular la saliencia basada en color y movimiento
    color_sm = color_SM(current_frame)
    motion_sm = motion_SM(prev_frame, current_frame)
    combined_sm = GPhase_color_SM(color_sm, motion_sm)

    # Convertir color_sm y motion_sm a float64
    color_sm = color_sm.astype('float64')
    motion_sm = motion_sm.astype('float64')

    # Visualización del mapa saliencia mediante color, mediante movimiento y ambos combinados
    subplot = cv2.hconcat((color_sm, motion_sm, combined_sm))
    ret, subplot = cv2.imencode('.jpg', subplot)
    #display_handle.update(Image(data=subplot.tobytes()))
    RG, BY, I, M = prueba(prev_frame, current_frame)
    channels = [RG, BY, I, M]
    results = []

    for c in channels:
        var = np.var(c)
        results.append(c / (var + 1e-8))

    weights = np.array(results) / np.sum(results, axis=0)
    combined_result = np.sum(weights * np.array(results), axis=0)

    cv2.imshow("RG", RG)
    cv2.imshow("BY", BY)
    cv2.imshow("I", I)
    cv2.imshow("M", M)
    cv2.imshow("Combined", combined_result)
    cv2.waitKey(10)
    """cv2.imshow("Color", color_sm)
    cv2.imshow("Motion", motion_sm)
    cv2.imshow("Color+Motion", combined_sm)"""

    if cv2.waitKey(20) == ord('q'):
        break

    # Actualizar el fotograma previo para el siguiente cálculo
    prev_frame = current_frame

# Liberar los recursos
cv2.destroyAllWindows()
video.release()
#display_handle.update(None)

## Gaussian Metrics Code

In [ ]:
# Definimos las listas para almacenar los valores de las métricas
AUCj_values = []
AUCshuff_values = []
NSS_values = []
CC_values = []

# Definimos los valores de sigma con un intervalo de 25
sigmas = [1, 25, 50, 75, 100, 125, 150, 175, 200]

# Iteramos sobre los diferentes valores de sigma
for sigma in sigmas:
    gaussian_AUCjudd_total = 0
    gaussian_AUCshuff_total = 0
    gaussian_NSS_total = 0
    gaussian_CC_total = 0
    gaussian_AUCjudd_scores = []
    gaussian_AUCshuff_scores = []
    gaussian_NSS_scores = []
    gaussian_CC_scores = []

    # Definimos parámetros y creamos la gaussiana
    shape = cv2.imread('./Imaxes/1.jpg', 1).shape
    gaussian_sm = CreateGaussian(shape, sigma)

    # Recorremos la base de datos con las imágenes iniciales
    for i in range(1, nimages+1):
        # Leemos la imagen y el mapa de fijación correspondiente
        image = cv2.imread(str(images_path)+str(i)+'.jpg', 0)
        ground_truth = fixation_maps['white'][0,i-1]*255
        random_ground_truth = fixation_maps['white'][0,np.random.randint(0,nimages)]*255

        # Calculamos las diferentes metricas
        AUCj_score = AUC_judd(gaussian_sm, ground_truth)
        gaussian_AUCjudd_scores.append(AUCj_score)
        gaussian_AUCjudd_total += AUCj_score

        AUCshuff_score = AUC_shuff(gaussian_sm, ground_truth, random_ground_truth)
        gaussian_AUCshuff_scores.append(AUCshuff_score)
        gaussian_AUCshuff_total += AUCshuff_score

        NSS_score = NSS(gaussian_sm, ground_truth)
        gaussian_NSS_scores.append(NSS_score)
        gaussian_NSS_total += NSS_score

        CC_score = CC(gaussian_sm, ground_truth)
        gaussian_CC_scores.append(CC_score)
        gaussian_CC_total += CC_score

    # Calculamos la media de las métricas y las almacenamos en las listas correspondientes
    AUCj_values.append(gaussian_AUCjudd_total/nimages)
    AUCshuff_values.append(gaussian_AUCshuff_total/nimages)
    NSS_values.append(gaussian_NSS_total/nimages)
    CC_values.append(gaussian_CC_total/nimages)

In [ ]:
# Creamos el DataFrame con las métricas
data = {
    'Sigma Value':sigmas,
    'AUCjudd':AUCj_values,
    'AUCshuff':AUCshuff_values,
    'NSS':NSS_values,
    'CC':CC_values
}

df = pd.DataFrame(data)

# Seleccionamos las columnas de interés para el gráfico de líneas
x = df['Sigma Value']
y1 = df['AUCjudd']
y2 = df['AUCshuff']
y3 = df['NSS']
y4 = df['CC']

# Creamos subplots separados para cada métrica
fig, axs = plt.subplots(4, 1, figsize=(10, 20))

# Graficamos cada métrica en su propio subplot
axs[0].plot(x, y1, marker='o', label='AUC judd', color='blue')
axs[0].set_title('AUC judd')
axs[0].set_xlabel('Valor sigma')
axs[0].set_ylabel('Valor métrica')
axs[0].grid(True)

axs[1].plot(x, y2, marker='o', label='AUC shuffled', color='orange')
axs[1].set_title('AUC shuffled')
axs[1].set_xlabel('Valor sigma')
axs[1].set_ylabel('Valor métrica')
axs[1].grid(True)

axs[2].plot(x, y3, marker='o', label='NSS', color='green')
axs[2].set_title('NSS')
axs[2].set_xlabel('Valor sigma')
axs[2].set_ylabel('Valor métrica')
axs[2].grid(True)

axs[3].plot(x, y4, marker='o', label='CC', color='red')
axs[3].set_title('CC')
axs[3].set_xlabel('Valor sigma')
axs[3].set_ylabel('Valor métrica')
axs[3].grid(True)

plt.tight_layout()
plt.show()

## WORKS FINE:

In [ ]:
def SR_Color_Multiescala(img, num_scales=1):
    rows, cols = img.shape[0], img.shape[1]

    # Si es una imagen en escala de grises (no se puede obtener información de color) 
    if len(img.shape) == 2:
        # Transformada de Fourier
        c = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
        mag = np.sqrt(c[:, :, 0] ** 2 + c[:, :, 1] ** 2)
        spectralResidual = np.exp(np.log(mag) - cv2.boxFilter(np.log(mag), -1, (3, 3)))

        # Aplicación del residuo espectral en el dominio de la frecuencia
        c[:, :, 0] = c[:, :, 0] * spectralResidual / mag
        c[:, :, 1] = c[:, :, 1] * spectralResidual / mag
        c = cv2.dft(c, flags=(cv2.DFT_INVERSE | cv2.DFT_SCALE))
        mag = c[:, :, 0] ** 2 + c[:, :, 1] ** 2

        # Suavizado
        sm = cv2.GaussianBlur(np.float32(mag),(85,85),20,20)

        # Redimensionamos y normalizamos el mapa de saliencia
        sm = cv2.normalize(cv2.resize(sm, (cols, rows), cv2.INTER_LANCZOS4), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        return sm

    # Si es una imagen RGB
    else:
        # Crear la pirámide gaussiana
        pyramid = [img]
        for _ in range(num_scales-1):
            img = cv2.pyrDown(img)
            pyramid.append(img)

        # Lista en la que se almacenarán los mapas de saliencia a distintas escalas
        smSS = []

        for scaled_img in pyramid:
            # Obtener los canales B, G, R
            B = scaled_img[:, :, 0]
            G = scaled_img[:, :, 1]
            R = scaled_img[:, :, 2]
            channels = [B, G, R]
            results = []

            # Para cada canal
            for i in range(len(channels)):
                # Transformada de Fourier para el canal actual
                c = cv2.dft(np.float32(channels[i]), flags=cv2.DFT_COMPLEX_OUTPUT)
                mag = np.sqrt(c[:, :, 0] ** 2 + c[:, :, 1] ** 2)
                spectralResidual = np.exp(np.log(mag) - cv2.boxFilter(np.log(mag), -1, (3, 3)))

                # Aplicación del residuo espectral al canal en el dominio de la frecuencia
                c[:, :, 0] = c[:, :, 0] * spectralResidual / mag
                c[:, :, 1] = c[:, :, 1] * spectralResidual / mag
                c = cv2.dft(c, flags=(cv2.DFT_INVERSE | cv2.DFT_SCALE))
                mag = c[:, :, 0] ** 2 + c[:, :, 1] ** 2

                # Suavizado
                sm_channel = cv2.GaussianBlur(np.float32(mag),(85,85),20,20)

                # Agregamos el mapa de saliencia del canal actual a la lista
                results.append(sm_channel)

            # Sumamos los mapas de saliencia de cada canal de color
            sm = np.sum(results, axis=0)

            # Introducimos el mapa de saliencia para la escala actual en la lista
            smSS.append(sm)

        # Reconstruir la imagen con las pirámides
        sm_ = smSS[0]
        for i in range(1, len(smSS)+1):
            smSS[-i] = cv2.resize(smSS[-i], (cols, rows))
            sm_ = np.add(sm_, smSS[-i])

        # Normalización
        sm_norm = cv2.normalize(sm_, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        return sm_norm

def PFT_Color_Multiescala(img, num_scales=1):
    rows, cols = img.shape[0], img.shape[1]

    # Si es una imagen en escala de grises, no podemos obtener información de los colores 
    if len(img.shape) == 2:
        # Transformada de Fourier
        img_fft = np.fft.fft2(img)

        # Espectro de la fase de la imagen
        phase_espectrum = np.angle(img_fft)

        # Mapa de saliencia
        sm = cv2.GaussianBlur(np.float32((np.abs(np.fft.ifft2(np.exp(1j * phase_espectrum)))**2)),(85,85),20,20)

        # Redimensionamos y normalizamos el mapa de saliencia
        sm = cv2.normalize(cv2.resize(sm, (cols, rows), cv2.INTER_LANCZOS4), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        return sm

    # Crear la pirámide gaussiana
    pyramid = [img]
    for _ in range(num_scales-1):
        img = cv2.pyrDown(img)
        pyramid.append(img)

    # Inicializar el mapa de saliencia final
    smSS = []

    for scaled_img in pyramid:
        # Obtener los canales B, G, R
        B = scaled_img[:, :, 0]
        G = scaled_img[:, :, 1]
        R = scaled_img[:, :, 2]
        channels = [B, G, R]
        results = []

        # Para cada canal
        for i in range(len(channels)):
            # Transformada de Fourier
            img_fft = np.fft.fft2(channels[i])

            # Espectro de la fase de la imagen
            phase_espectrum = np.angle(img_fft)

            # Suavizado
            sm_channel = cv2.GaussianBlur(np.float32((np.abs(np.fft.ifft2(np.exp(1j * phase_espectrum)))**2)),(85,85),20,20)

            # Agregamos el mapa de saliencia del canal actual a la lista
            results.append(sm_channel)

        # Sumamos los mapas de saliencia de cada canal de color
        sm = np.sum(results, axis=0)

        # Introducimos el mapa de saliencia para la escala actual en la lista
        smSS.append(sm)

    # Reconstruir la imagen con las pirámides
    sm_ = smSS[0]
    for i in range(1, len(smSS)+1):
        smSS[-i] = cv2.resize(smSS[-i], (cols, rows))
        sm_ = np.add(sm_, smSS[-i])

    # Normalización
    sm_norm = cv2.normalize(cv2.resize(sm_, (cols, rows), cv2.INTER_LANCZOS4), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return sm_norm

# DINÁMICA

## OLD PFT_VIDEO

In [ ]:
# Función para calcular el mapa de saliencia basado en el espectro de fase de la transformada de Fourier para video
def PFT_video(prev_frame, current_frame):
    # Si es una imagen en escala de grises, no podemos obtener información de los colores 
    if len(current_frame.shape) == 2:
        print("Imagen en escala de grises, devolviendo la imagen original...")
        return

    rows, cols = current_frame.shape[0], current_frame.shape[1]

    b, g, r = cv2.split(current_frame)

    # Canal de color
    R = r - (g + b)/2
    G = g - (r + b)/2
    B = b - (r + g)/2
    Y = (r + g)/2 - np.abs(r - g)/2 - b

    # Canales componente-opuesta de color
    RG = R - G
    BY = B - Y

    # Canal de intensidad
    I = np.sum(current_frame, axis=-1)

    # Canal de motion
    prev_I = np.sum(prev_frame, axis=-1)
    motion = np.abs(I-prev_I)

    # Representación de q(t) en forma simplectica
    # f1(t):
    planes = [motion.astype(np.float64), RG.astype(np.float64)]
    f1 = cv2.dft(cv2.merge(planes))
    planes = cv2.split(f1)

    # Magnitud del espectro1
    mag1 = cv2.magnitude(planes[0], planes[1])
    mag1 = cv2.multiply(mag1, mag1)

    # f2(t):
    planes = [BY.astype(np.float64), I.astype(np.float64)]
    f2 = cv2.dft(cv2.merge(planes))
    planes = cv2.split(f2)

    # Magnitud del espectro2
    mag2 = cv2.magnitude(planes[0], planes[1])
    mag2 = cv2.multiply(mag2, mag2)

    # Combinamos las magnitudes
    mag = cv2.sqrt(mag1 + mag2)

    # Normalizar
    # f1(t):
    planes = list(cv2.split(f1))
    planes[0] = planes[0]/mag
    planes[1] = planes[1]/mag
    f1 = cv2.merge(planes)

    # f2(t):
    planes = list(cv2.split(f2))
    planes[0] = planes[0]/mag
    planes[1] = planes[1]/mag
    f2 = cv2.merge(planes)

    # Transformada inversa de Fourier
    cv2.dft(f1, f1, cv2.DFT_INVERSE)
    cv2.dft(f2, f2, cv2.DFT_INVERSE)

    # Magnitud de la tranformada inversa de Fourier
    planes = cv2.split(f1)
    mag1 = cv2.magnitude(planes[0], planes[1])
    mag1 = cv2.multiply(mag1, mag1)

    planes = cv2.split(f2)
    mag2 = cv2.magnitude(planes[0], planes[1])
    mag2 = cv2.multiply(mag2, mag2)

    # Magnitud total
    mag = cv2.sqrt(mag1 + mag2)

    # Suavizado de la imagen
    mag = cv2.GaussianBlur(mag, (5, 5), 8, None, 8)

    # Normalizar el mapa de saliencia
    sm = cv2.normalize(mag, np.zeros((rows, cols, 1), np.uint8), 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return sm


# Cargamos los videos
videos_list = os.listdir("./videos")
for i in range(len(videos_list)):
    cap = cv2.VideoCapture("./videos/"+videos_list[i])

    # Leer el primer fotograma
    ret, prev_frame = cap.read()

    # Iterar sobre los fotogramas del video
    while cap.isOpened():
        ret, current_frame = cap.read()
        if not ret:
            break

        # Llamamos a la función para calcular el mapa de saliencia a multiescala con información de color e invariante a la escala
        sm = PFT_video(prev_frame, current_frame)

        # Visualización
        cv2.imshow('Saliency map {}'.format(videos_list[i]), sm)

        # Actualizamos
        prev_frame = current_frame

        if cv2.waitKey(15) & 0xFF == ord('q'):
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

## OLD PDZHOU+COLOR

In [ ]:
# Función para calcular el mapa de saliencia mediante la discrepancia de la fase
def PDZhou_color_SM(prev_frame, current_frame):
    # Obtenemos la dimensión de los frames
    shape_frame = (prev_frame.shape[1], prev_frame.shape[0])    # (width, height)

    # Mapa de calor por colores:
    # Convertir el fotograma a espacio de color LAB
    lab_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2LAB)

    # Separar los canales LAB
    l, a, b = cv2.split(lab_frame)

    # Normalizar los canales a valores entre 0 y 1
    l_normalized = cv2.normalize(l, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    a_normalized = cv2.normalize(a, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    b_normalized = cv2.normalize(b, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Calcular la saliencia basada en todos los canales LAB
    sm_c = cv2.GaussianBlur(l_normalized, (5, 5), sigmaX=8, sigmaY=8)
    sm_c += cv2.GaussianBlur(a_normalized, (5, 5), sigmaX=8, sigmaY=8)
    sm_c += cv2.GaussianBlur(b_normalized, (5, 5), sigmaX=8, sigmaY=8)

    # Escalar a valores entre 0 y 255 y luego invertir los colores
    sm_c = 255 - (sm_c * 255 / np.max(sm_c))

    # Mapa de calor por movimiento:
    # Escalar y redimensionar los fotogramas de entrada
    prev_frame = cv2.resize(cv2.normalize(prev_frame, None, 0.0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F), (320, 240))
    current_frame = cv2.resize(cv2.normalize(current_frame, None, 0.0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F), (320, 240))

    # Calcular la transformada de Fourier de los fotogramas
    prev_FFT = np.fft.fft2(prev_frame)
    current_FFT = np.fft.fft2(current_frame)

    # Calcular la magnitud y fase de las transformadas de Fourier
    prev_mag = np.abs(prev_FFT)
    current_mag = np.abs(current_FFT)
    prev_phase = np.angle(prev_FFT)
    current_phase = np.angle(current_FFT)

    # Calcular los mapas de magnitud usando la discrepancia de fase
    mMap1 = np.abs(np.fft.ifft2((current_mag - prev_mag) * np.exp(1j * prev_phase)))
    mMap2 = np.abs(np.fft.ifft2((current_mag - prev_mag) * np.exp(1j * current_phase)))

    # Multiplicar los mapas de magnitud para obtener la imagen de saliencia
    img = mMap1 * mMap2

    # Sumar sobre los canales de color para obtener un solo mapa de saliencia
    sm_m = np.sum(img, axis=2)

    # Media de ambos mapas de calor:
    sm = (sm_c*0.01 + sm_m*100) / 2 # Le quitamos peso a la información de color y se lo damos al movimiento para nivelarlos

    # Redimensionar el mapa de saliencia a la forma deseada
    sm_resized = cv2.resize(sm, shape_frame)
    
    # Normalizar el mapa de saliencia para el rango 0-255
    sm_norm = cv2.normalize(sm_resized, None, 0.0, 255, cv2.NORM_MINMAX, cv2.CV_32F)

    return sm_norm


# Leer el video
video = cv2.VideoCapture('./videos/SCE_bur_1_distractores_altos.avi')

# Leer el primer fotograma
ret, prev_frame = video.read()
display_handle = display(None, display_id=True)
while True:
    ret, current_frame = video.read()
    if not ret:
        break

    # Calcular la saliencia
    sm = PDZhou_color_SM(prev_frame, current_frame)

    # Visualización del mapa saliencia
    ret, sm = cv2.imencode('.jpg', sm)
    display_handle.update(Image(data=sm.tobytes()))
    cv2.waitKey(10)

    # Actualizar el fotograma previo para el siguiente cálculo
    prev_frame = current_frame

# Liberar los recursos
video.release()
display_handle.update(None)

## PFT (VIDEO) INCORRECTO (2 FRAMES)

In [ ]:
WIDTH = 64

# Función para calcular el mapa de saliencia basado en el espectro de fase de la transformada de Fourier
def PFTVideo_Color_Multiescala(previous_frame, current_frame, num_scales=1):
    if len(previous_frame.shape) == 2:
        rows, cols = previous_frame.shape
    else:
        rows, cols, _ = previous_frame.shape

    previous_frame = cv2.resize(previous_frame, (WIDTH, int(WIDTH * previous_frame.shape[0] / previous_frame.shape[1])))
    current_frame = cv2.resize(current_frame, (WIDTH, int(WIDTH * current_frame.shape[0] / current_frame.shape[1])))

    # Crear la pirámide gaussiana para previous_frame
    previous_pyramid = [previous_frame]
    for _ in range(num_scales - 1):
        previous_frame = cv2.pyrDown(previous_frame)
        previous_pyramid.append(previous_frame)

    # Crear la pirámide gaussiana para current_frame
    current_pyramid = [current_frame]
    for _ in range(num_scales - 1):
        current_frame = cv2.pyrDown(current_frame)
        current_pyramid.append(current_frame)

    # Lista en la que se almacenarán los mapas de saliencia a distintas escalas
    smSS = []

    for prev, curr in zip(previous_pyramid, current_pyramid):
        # Si son imágenes en escala de grises, no podemos obtener información de los colores
        if len(prev.shape) == 2:
            # Calculamos la diferencia entre los frames
            diff = cv2.absdiff(prev, curr)

            # Transformada de Fourier
            diff_fft = np.fft.fft2(diff)

            # Espectro de la fase de la imagen
            phase_spectrum = np.angle(diff_fft)

            # Suavizado y redimensionado del mapa de saliencia
            sm = cv2.resize(cv2.GaussianBlur((np.abs(np.fft.ifft2(np.exp(1j * phase_spectrum))) ** 2),(11,11),3,3),(cols, rows), cv2.NORM_MINMAX)

            # Introducimos el mapa de saliencia para la escala actual en la lista
            smSS.append(sm)

        # Si son imágenes a color
        else:
            # Obtener los canales B, G, R
            B_prev, G_prev, R_prev = prev[:, :, 0], prev[:, :, 1], prev[:, :, 2]
            B_curr, G_curr, R_curr = curr[:, :, 0], curr[:, :, 1], curr[:, :, 2]

            # Calculamos la diferencia entre los frames para cada canal
            diff_B = cv2.absdiff(B_prev, B_curr)
            diff_G = cv2.absdiff(G_prev, G_curr)
            diff_R = cv2.absdiff(R_prev, R_curr)

            channels_diff = [diff_B, diff_G, diff_R]
            results = []

            # Para cada canal de diferencia
            for i in range(len(channels_diff)):
                # Transformada de Fourier
                diff_fft = np.fft.fft2(channels_diff[i])

                # Espectro de la fase de la imagen
                phase_spectrum = np.angle(diff_fft)

                # Suavizado y redimensionado del mapa de saliencia para el canal actual
                sm_channel = cv2.resize(cv2.GaussianBlur((np.abs(np.fft.ifft2(np.exp(1j * phase_spectrum))) ** 2),(11,11),3,3), (cols, rows), cv2.NORM_MINMAX)

                # Agregamos el mapa de saliencia del canal actual a la lista
                results.append(sm_channel)

            # Sumamos los mapas de saliencia de cada canal de color
            sm = np.sum(results, axis=0)

            # Introducimos el mapa de saliencia para la escala actual en la lista
            smSS.append(sm)

    # Reconstruir la imagen con las pirámides
    sm_ = smSS[0]
    for i in range(1, len(smSS) + 1):
        smSS[-i] = cv2.resize(smSS[-i], (cols, rows), cv2.NORM_MINMAX)
        sm_ = np.add(sm_, smSS[-i]) 

    # Normalización
    sm_norm = cv2.normalize(sm_, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return sm_norm


# Cargamos los videos
videos_list = os.listdir("./videos")
for i in range(len(videos_list)):
    video = cv2.VideoCapture("./videos/"+videos_list[i])

    # Leer el primer fotograma
    ret, prev_frame = video.read()

    # Iterar sobre los fotogramas del video
    while video.isOpened():
        ret, current_frame = video.read()
        if not ret:
            break

        # Llamamos a la función para calcular el mapa de saliencia a multiescala con información de color e invariante a la escala
        sm = PFTVideo_Color_Multiescala(prev_frame, current_frame, num_scales=1)

        # Visualización
        cv2.imshow('Saliency map {}'.format(videos_list[i]), sm)

        # Actualizamos
        prev_frame = current_frame

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    # Liberar recursos
    video.release()
    cv2.destroyAllWindows()

## OLD METRICS

In [ ]:
gp_AUC_scores = []
gp_NSS_scores = []
gp_CC_scores = []
pd_AUCjudd_scores = []
pd_AUCshuff_scores = []
pd_NSS_scores = []
pd_CC_scores = []


# Recorremos la base de datos con los videos iniciales
for ivideo in range(fixation_maps['EyeTrackVDB'].shape[0]):
    print(ivideo)
    # Leemos el video y el mapa de fijación correspondiente
    name_video = fixation_maps['EyeTrackVDB'][ivideo]['Name'][0][0]
    video = cv2.VideoCapture(videos_path + name_video)

    if (video.isOpened() == False):
        print("Error en la apertura del fichero {}".format(videos_path + name_video))

    # Recuperamos as propieades do video.
    frame_w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_count = 0
    ret, prev_frame = video.read()
    while(True):
        ret, current_frame = video.read()
        if(not ret):
            break

        # Obtenemos el mapa de saliencia para el frame
        #gp_sm_frame = PFTVideo_Color_Multiescala(prev_frame, current_frame)  # Fase global
        pd_sm_frame = PDZhou(prev_frame, current_frame)  # Discrepancia de la fase

        # Leer las fijaciones del frame
        fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,frame_count][:,0:2]
        other_fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,np.random.randint(len(fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0]))][:,0:2]

        # Construir el mapa de densidad de fijaciones
        map_fix_frame = np.zeros((frame_h,frame_w))
        for ifix in range(fix.shape[0]):
            # Puede haber fijaciones fuera de los margenes de la imagen
            if(int(fix[ifix][0]) < frame_h and int(fix[ifix][1]) < frame_w):
                map_fix_frame[int(fix[ifix][1]), int(fix[ifix][0])] = 255   # Mapa de fijaciones
        # Convolucionamos con una gaussiana las fijaciones para obtener el mapa de densidad de fijaciones
        map_den_fix = (np.floor(cv2.normalize(cv2.GaussianBlur(np.float32(map_fix_frame),(85,85),20,20), None, 0, 255, cv2.NORM_MINMAX))).astype(np.uint8)

        # Construir otro mapa de densidad de fijaciones
        other_map_fix_frame = np.zeros((frame_h,frame_w))
        for ifix in range(other_fix.shape[0]):
            # Puede haber fijaciones fuera de los margenes de la imagen
            if(int(other_fix[ifix][0]) < frame_h and int(other_fix[ifix][1]) < frame_w):
                other_map_fix_frame[int(other_fix[ifix][1]), int(other_fix[ifix][0])] = 255 # Mapa de fijaciones
        # Convolucionamos con una gaussiana las fijaciones para obtener el mapa de densidad de fijaciones
        other_map_den_fix = (np.floor(cv2.normalize(cv2.GaussianBlur(np.float32(other_map_fix_frame),(85,85),20,20), None, 0, 255, cv2.NORM_MINMAX))).astype(np.uint8)


        # Calculamos las diferentes métricas para cada frame
        # Fase global:
        """gp_AUC_score_frame = AUC_shuff(gp_sm_frame, map_den_fix, other_map_den_fix)
        gp_AUC_scores.append(gp_AUC_score_frame)

        gp_NSS_score_frame = NSS(gp_sm_frame, map_den_fix)
        gp_NSS_scores.append(gp_NSS_score_frame)

        gp_CC_score_frame = CC(gp_sm_frame, map_den_fix)
        gp_CC_scores.append(gp_CC_score_frame)"""

        # Discrepancia de la fase:
        pd_AUCjudd_scores.append(AUC_judd(pd_sm_frame, map_den_fix))

        pd_AUCshuff_scores.append(AUC_shuff(pd_sm_frame, map_den_fix, other_map_den_fix))

        pd_NSS_scores.append(NSS(pd_sm_frame, map_den_fix))

        pd_CC_scores.append(CC(pd_sm_frame, map_den_fix))

        # Actualizamos los frames
        frame_count += 1
        prev_frame = current_frame

# Calculamos la media de la métricas
"""gp_AUC_mean = np.mean(np.nan_to_num(gp_AUC_scores))
gp_NSS_mean = np.mean(np.nan_to_num(gp_NSS_scores))
gp_CC_mean = np.mean(np.nan_to_num(gp_CC_scores))"""
pd_AUCjudd_mean = np.mean(np.nan_to_num(pd_AUCjudd_scores))
pd_AUCshuff_mean = np.mean(np.nan_to_num(pd_AUCshuff_scores))
pd_NSS_mean = np.mean(np.nan_to_num(pd_NSS_scores))
pd_CC_mean = np.mean(np.nan_to_num(pd_CC_scores))

# TEST

In [2]:
import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import os
import time
from IPython.display import display, Image

def PDZhou(prev_frame, current_frame):
    # Si el video está en escala de grises
    if len(prev_frame) == 2:
        # Calculamos la Transformada de Fourier 2D para cada fotograma
        FFT1 = np.fft.fft2(prev_frame)
        FFT2 = np.fft.fft2(current_frame)

        # Calculamos la magnitud y la fase de cada transformada de Fourier
        Amp1 = np.abs(FFT1)
        Amp2 = np.abs(FFT2)
        Phase1 = np.angle(FFT1)
        Phase2 = np.angle(FFT2)

        # Calculamos los mapas de magnitud
        mMap1 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase1)))
        mMap2 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase2)))

        # Calculamos el mapa combinado
        mMap = mMap1 * mMap2

        # Suavizado
        mMap = cv2.GaussianBlur(mMap,(85,85),20,20)

    # Si tiene canales de color
    else:
        # Obtenemos los canales de color
        prev_frame_channels = [prev_frame[:, :, 0], prev_frame[:, :, 1], prev_frame[:, :, 2]]
        current_frame_channels = [current_frame[:, :, 0], current_frame[:, :, 1], current_frame[:, :, 2]]
        results = []

        # Calculamos el mapa de saliencia para cada canal de color
        for i in range(len(prev_frame_channels)):
            # Calculamos la Transformada de Fourier 2D para cada fotograma
            FFT1 = np.fft.fft2(prev_frame_channels[i])
            FFT2 = np.fft.fft2(current_frame_channels[i])

            # Calculamos la magnitud y la fase de cada transformada de Fourier
            Amp1 = np.abs(FFT1)
            Amp2 = np.abs(FFT2)
            Phase1 = np.angle(FFT1)
            Phase2 = np.angle(FFT2)

            # Calculamos los mapas de magnitud
            mMap1 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase1)))
            mMap2 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase2)))

            # Calculamos el mapa combinado
            mMap_channel = mMap1 * mMap2

            # Suavizado
            mMap_channel = cv2.GaussianBlur(mMap_channel,(85,85),20,20)

            # Agregamos el mapa de saliencia del canal actual a la lista
            results.append(mMap_channel)

        # Sumamos los mapas de saliencia de cada canal de color
        mMap = np.sum(results, axis=0)

    # Normalización
    mMap_norm = cv2.normalize(mMap, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return mMap_norm



def normalize_map(s_map):
	# normalize the salience map
	norm_s_map = (s_map - np.min(s_map))/((np.max(s_map)-np.min(s_map))*1.0)
	return norm_s_map

def discretize_gt(gt):
	return gt/255

def AUC_judd(s_map,gt):
	# ground truth is discrete, s_map is continous and normalized
	gt = discretize_gt(gt)
	s_map = normalize_map(s_map)
	# thresholds are calculated from the salience map, only at places where fixations are present
	thresholds = []
	for i in range(0,gt.shape[0]):
		for k in range(0,gt.shape[1]):
			if gt[i][k]>0:
				thresholds.append(s_map[i][k])

	num_fixations = np.sum(gt)
	# num fixations is no. of salience map values at gt >0
	thresholds = sorted(set(thresholds))
	#fp_list = []
	#tp_list = []
	area = []
	area.append((0.0,0.0))
	for thresh in thresholds:
		# in the salience map, keep only those pixels with values above threshold
		temp = np.zeros(s_map.shape)
		temp[s_map>=thresh] = 1.0
		num_overlap = np.where(np.add(temp,gt)==2)[0].shape[0]
		tp = num_overlap/(num_fixations*1.0)
		
		# total number of pixels > threshold - number of pixels that overlap with gt / total number of non fixated pixels
		# this becomes nan when gt is full of fixations..this won't happen
		fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
		
		area.append((round(tp,4),round(fp,4)))
		#tp_list.append(tp)
		#fp_list.append(fp)

	#tp_list.reverse()
	#fp_list.reverse()
	area.append((1.0,1.0))
	#tp_list.append(1.0)
	#fp_list.append(1.0)
	#print tp_list
	area.sort(key = lambda x:x[0])
	tp_list =  [x[0] for x in area]
	fp_list =  [x[1] for x in area]
	return np.trapz(np.array(tp_list),np.array(fp_list))

def AUC_shuff(s_map,gt,other_map,splits=100,stepsize=0.1):
	gt = discretize_gt(gt)
	other_map = discretize_gt(other_map)
	s_map = normalize_map(s_map)

	num_fixations = np.sum(gt)
	
	x,y = np.where(other_map==1)
	other_map_fixs = []
	for j in zip(x,y):
		other_map_fixs.append(j[0]*other_map.shape[0] + j[1])
	ind = len(other_map_fixs)
	assert ind == len(other_map_fixs), 'something is wrong in auc shuffle'


	num_fixations_other = min(ind,num_fixations)

	num_pixels = s_map.shape[0]*s_map.shape[1]
	random_numbers = []
	for i in range(0,splits):
		temp_list = []
		t1 = np.random.permutation(ind)
		for k in t1:
			temp_list.append(other_map_fixs[k])
		random_numbers.append(temp_list)	

	aucs = []
	# for each split, calculate auc
	for i in random_numbers:
		r_sal_map = []
		for k in i:
			#print('{}  {}'.format(k%s_map.shape[0]-1, int(np.floor(k/s_map.shape[0]))))
			r_sal_map.append(s_map[k%s_map.shape[0]-1, int(np.floor(k/s_map.shape[0]))])
		# in these values, we need to find thresholds and calculate auc
		thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

		r_sal_map = np.array(r_sal_map)

		# once threshs are got
		thresholds = sorted(set(thresholds))
		area = []
		area.append((0.0,0.0))
		for thresh in thresholds:
			# in the salience map, keep only those pixels with values above threshold
			temp = np.zeros(s_map.shape)
			temp[s_map>=thresh] = 1.0
			num_overlap = np.where(np.add(temp,gt)==2)[0].shape[0]
			tp = num_overlap/(num_fixations*1.0)
			
			#fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
			# number of values in r_sal_map, above the threshold, divided by num of random locations = num of fixations
			fp = len(np.where(r_sal_map>thresh)[0])/(num_fixations*1.0)

			area.append((round(tp,4),round(fp,4)))
		
		area.append((1.0,1.0))
		area.sort(key = lambda x:x[0])
		tp_list =  [x[0] for x in area]
		fp_list =  [x[1] for x in area]

		aucs.append(np.trapz(np.array(tp_list),np.array(fp_list)))
	
	return np.mean(aucs)

def NSS(s_map,gt):
	gt = discretize_gt(gt)
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)

	x,y = np.where(gt==1)
	temp = []
	for i in zip(x,y):
		temp.append(s_map_norm[i[0],i[1]])
	return np.mean(temp)


def CC(s_map,gt):
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)
	gt_norm = (gt - np.mean(gt))/np.std(gt)
	a = s_map_norm
	b= gt_norm
	r = (a*b).sum() / np.sqrt((a*a).sum() * (b*b).sum())
	return r




# Definimos el path a los videos
videos_path = './videos/'

# Cargamos los mapas de fijación
fixation_maps =  sio.loadmat('./FixVideoData.mat')

# Definimos el número de videos a procesar
nvideos = 6

# Recorremos la base de datos con los videos iniciales
for ivideo in range(fixation_maps['EyeTrackVDB'].shape[0]):
    # Leemos el video y el mapa de fijación correspondiente
    name_video = fixation_maps['EyeTrackVDB'][ivideo]['Name'][0][0]
    video = cv2.VideoCapture(videos_path + name_video)

    if (video.isOpened() == False):
        print("Error en la apertura del fichero {}".format(videos_path + name_video))

    # Recuperamos as propieades do video.
    frame_w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_count = 0
    ret, prev_frame = video.read()
    while(True):
        ret, current_frame = video.read()
        if(not ret):
            break

        # Obtenemos el mapa de saliencia para el frame
        #gp_sm_frame = PFTVideo_Color_Multiescala(prev_frame, current_frame)  # Fase global
        pd_sm_frame = PDZhou(prev_frame, current_frame)  # Discrepancia de la fase

        # Leer las fijaciones del frame
        fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,frame_count][:,0:2]
        randNum = np.random.randint(len(fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0]))
        print(randNum)
        other_fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,randNum][:,0:2]

        # Construir el mapa de densidad de fijaciones
        map_fix_frame = np.zeros((frame_h,frame_w))
        for ifix in range(fix.shape[0]):
            # Puede haber fijaciones fuera de los margenes de la imagen
            if(int(fix[ifix][0]) < frame_h and int(fix[ifix][1]) < frame_w):
                map_fix_frame[int(fix[ifix][1]), int(fix[ifix][0])] = 255   # Mapa de fijaciones
        # Convolucionamos con una gaussiana las fijaciones para obtener el mapa de densidad de fijaciones
        map_den_fix = (np.floor(cv2.normalize(cv2.GaussianBlur(np.float32(map_fix_frame),(85,85),20,20), None, 0, 255, cv2.NORM_MINMAX))).astype(np.uint8)

        # Visualización
        cv2.imshow('Saliency map {}'.format(randNum), map_den_fix)
        time.sleep(5)

        # Actualizamos
        prev_frame = current_frame

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

/home/adrian/.virtualenvs/VAA/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/adrian/.virtualenvs/VAA/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/adrian/.virtualenvs/VAA/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/adrian/.virtualenvs/VAA/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


129
59
86


: 

: 

In [148]:
import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import os
import time
from IPython.display import display, Image

def PDZhou(prev_frame, current_frame):
    # Si el video está en escala de grises
    if len(prev_frame) == 2:
        # Calculamos la Transformada de Fourier 2D para cada fotograma
        FFT1 = np.fft.fft2(prev_frame)
        FFT2 = np.fft.fft2(current_frame)

        # Calculamos la magnitud y la fase de cada transformada de Fourier
        Amp1 = np.abs(FFT1)
        Amp2 = np.abs(FFT2)
        Phase1 = np.angle(FFT1)
        Phase2 = np.angle(FFT2)

        # Calculamos los mapas de magnitud
        mMap1 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase1)))
        mMap2 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase2)))

        # Calculamos el mapa combinado
        mMap = mMap1 * mMap2

        # Suavizado
        mMap = cv2.GaussianBlur(mMap,(85,85),20,20)

    # Si tiene canales de color
    else:
        # Obtenemos los canales de color
        prev_frame_channels = [prev_frame[:, :, 0], prev_frame[:, :, 1], prev_frame[:, :, 2]]
        current_frame_channels = [current_frame[:, :, 0], current_frame[:, :, 1], current_frame[:, :, 2]]
        results = []

        # Calculamos el mapa de saliencia para cada canal de color
        for i in range(len(prev_frame_channels)):
            # Calculamos la Transformada de Fourier 2D para cada fotograma
            FFT1 = np.fft.fft2(prev_frame_channels[i])
            FFT2 = np.fft.fft2(current_frame_channels[i])

            # Calculamos la magnitud y la fase de cada transformada de Fourier
            Amp1 = np.abs(FFT1)
            Amp2 = np.abs(FFT2)
            Phase1 = np.angle(FFT1)
            Phase2 = np.angle(FFT2)

            # Calculamos los mapas de magnitud
            mMap1 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase1)))
            mMap2 = np.abs(np.fft.ifft2((Amp2 - Amp1) * np.exp(1j * Phase2)))

            # Calculamos el mapa combinado
            mMap_channel = mMap1 * mMap2

            # Suavizado
            mMap_channel = cv2.GaussianBlur(mMap_channel,(85,85),20,20)

            # Agregamos el mapa de saliencia del canal actual a la lista
            results.append(mMap_channel)

        # Sumamos los mapas de saliencia de cada canal de color
        mMap = np.sum(results, axis=0)

    # Normalización
    mMap_norm = cv2.normalize(mMap, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return mMap_norm



def normalize_map(s_map):
	# normalize the salience map
	norm_s_map = (s_map - np.min(s_map))/((np.max(s_map)-np.min(s_map))*1.0)
	return norm_s_map

def discretize_gt(gt):
	return gt/255

def AUC_judd(s_map,gt):
	# ground truth is discrete, s_map is continous and normalized
	gt = discretize_gt(gt)
	s_map = normalize_map(s_map)
	# thresholds are calculated from the salience map, only at places where fixations are present
	thresholds = []
	for i in range(0,gt.shape[0]):
		for k in range(0,gt.shape[1]):
			if gt[i][k]>0:
				thresholds.append(s_map[i][k])

	num_fixations = np.sum(gt)
	# num fixations is no. of salience map values at gt >0
	thresholds = sorted(set(thresholds))
	#fp_list = []
	#tp_list = []
	area = []
	area.append((0.0,0.0))
	for thresh in thresholds:
		# in the salience map, keep only those pixels with values above threshold
		temp = np.zeros(s_map.shape)
		temp[s_map>=thresh] = 1.0
		num_overlap = np.where(np.add(temp,gt)==2)[0].shape[0]
		tp = num_overlap/(num_fixations*1.0)
		
		# total number of pixels > threshold - number of pixels that overlap with gt / total number of non fixated pixels
		# this becomes nan when gt is full of fixations..this won't happen
		fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
		
		area.append((round(tp,4),round(fp,4)))
		#tp_list.append(tp)
		#fp_list.append(fp)

	#tp_list.reverse()
	#fp_list.reverse()
	area.append((1.0,1.0))
	#tp_list.append(1.0)
	#fp_list.append(1.0)
	#print tp_list
	area.sort(key = lambda x:x[0])
	tp_list =  [x[0] for x in area]
	fp_list =  [x[1] for x in area]
	return np.trapz(np.array(tp_list),np.array(fp_list))

def AUC_shuff(s_map,gt,other_map,splits=100,stepsize=0.1):
	gt = discretize_gt(gt)
	other_map = discretize_gt(other_map)
	s_map = normalize_map(s_map)

	num_fixations = np.sum(gt)
	
	x,y = np.where(other_map==1)
	other_map_fixs = []
	for j in zip(x,y):
		other_map_fixs.append(j[0]*other_map.shape[0] + j[1])
	ind = len(other_map_fixs)
	assert ind == len(other_map_fixs), 'something is wrong in auc shuffle'


	num_fixations_other = min(ind,num_fixations)

	num_pixels = s_map.shape[0]*s_map.shape[1]
	random_numbers = []
	for i in range(0,splits):
		temp_list = []
		t1 = np.random.permutation(ind)
		for k in t1:
			temp_list.append(other_map_fixs[k])
		random_numbers.append(temp_list)	

	aucs = []
	# for each split, calculate auc
	for i in random_numbers:
		r_sal_map = []
		for k in i:
			#print('{}  {}'.format(k%s_map.shape[0]-1, int(np.floor(k/s_map.shape[0]))))
			r_sal_map.append(s_map[k%s_map.shape[0]-1, int(np.floor(k/s_map.shape[0]))])
		# in these values, we need to find thresholds and calculate auc
		thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

		r_sal_map = np.array(r_sal_map)

		# once threshs are got
		thresholds = sorted(set(thresholds))
		area = []
		area.append((0.0,0.0))
		for thresh in thresholds:
			# in the salience map, keep only those pixels with values above threshold
			temp = np.zeros(s_map.shape)
			temp[s_map>=thresh] = 1.0
			num_overlap = np.where(np.add(temp,gt)==2)[0].shape[0]
			tp = num_overlap/(num_fixations*1.0)
			
			#fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
			# number of values in r_sal_map, above the threshold, divided by num of random locations = num of fixations
			fp = len(np.where(r_sal_map>thresh)[0])/(num_fixations*1.0)

			area.append((round(tp,4),round(fp,4)))
		
		area.append((1.0,1.0))
		area.sort(key = lambda x:x[0])
		tp_list =  [x[0] for x in area]
		fp_list =  [x[1] for x in area]

		aucs.append(np.trapz(np.array(tp_list),np.array(fp_list)))
	
	return np.mean(aucs)

def NSS(s_map,gt):
	gt = discretize_gt(gt)
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)

	x,y = np.where(gt==1)
	temp = []
	for i in zip(x,y):
		temp.append(s_map_norm[i[0],i[1]])
	return np.mean(temp)


def CC(s_map,gt):
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)
	gt_norm = (gt - np.mean(gt))/np.std(gt)
	a = s_map_norm
	b= gt_norm
	r = (a*b).sum() / np.sqrt((a*a).sum() * (b*b).sum())
	return r


# Definimos el path a los videos
videos_path = './videos/'

# Cargamos los mapas de fijación
fixation_maps =  sio.loadmat('./FixVideoData.mat')

# Definimos el número de videos a procesar
nvideos = 6

# Recuperamos as propieades do video.
frame_w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_count = 0
for ivideo in range(fixation_maps['EyeTrackVDB'].shape[0]):
	name_video = fixation_maps['EyeTrackVDB'][ivideo]['Name'][0][0]
	video = cv2.VideoCapture(videos_path + name_video)

	print(len(fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0]))

  


	# count the number of frames 
	frames = video.get(cv2.CAP_PROP_FRAME_COUNT) 
	ret, frame = video.read()
	print(frames)
"""ret, prev_frame = video.read()
while(True):
    ret, current_frame = video.read()
    if(not ret):
        break

    # Obtenemos el mapa de saliencia para el frame
    #gp_sm_frame = PFTVideo_Color_Multiescala(prev_frame, current_frame)  # Fase global
    #pd_sm_frame = PDZhou(prev_frame, current_frame)  # Discrepancia de la fase

    # Leer las fijaciones del frame
    fix = fixation_maps['EyeTrackVDB'][0]['Frame'][0]['FixL'][0,frame_count][:,0:2]
    
    print(fixation_maps['EyeTrackVDB'][0]['Frame'][0]['FixL'][0,frame_count][:,0:2])

    randNum = np.random.randint(len(fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0]))
    print(randNum)
    other_fix = fixation_maps['EyeTrackVDB'][0]['Frame'][0]['FixL'][0,randNum][:,0:2]

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break"""

"""if (video.isOpened() == False):
    print("Error en la apertura del fichero {}".format(videos_path + name_video))

# Recuperamos as propieades do video.
frame_w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_count = 0
ret, prev_frame = video.read()
while(True):
    ret, current_frame = video.read()
    if(not ret):
        break

    # Obtenemos el mapa de saliencia para el frame
    #gp_sm_frame = PFTVideo_Color_Multiescala(prev_frame, current_frame)  # Fase global
    pd_sm_frame = PDZhou(prev_frame, current_frame)  # Discrepancia de la fase

    # Leer las fijaciones del frame
    fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,frame_count][:,0:2]
    randNum = np.random.randint(len(fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0]))
    print(randNum)
    other_fix = fixation_maps['EyeTrackVDB'][ivideo]['Frame'][0]['FixL'][0,randNum][:,0:2]

    # Construir el mapa de densidad de fijaciones
    map_fix_frame = np.zeros((frame_h,frame_w))
    for ifix in range(fix.shape[0]):
        # Puede haber fijaciones fuera de los margenes de la imagen
        if(int(fix[ifix][0]) < frame_h and int(fix[ifix][1]) < frame_w):
            map_fix_frame[int(fix[ifix][1]), int(fix[ifix][0])] = 255   # Mapa de fijaciones
    # Convolucionamos con una gaussiana las fijaciones para obtener el mapa de densidad de fijaciones
    map_den_fix = (np.floor(cv2.normalize(cv2.GaussianBlur(np.float32(map_fix_frame),(85,85),20,20), None, 0, 255, cv2.NORM_MINMAX))).astype(np.uint8)

    # Visualización
    cv2.imshow('Saliency map {}'.format(randNum), map_den_fix)
    time.sleep(5)

    # Actualizamos
    prev_frame = current_frame

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break"""

# Liberar recursos
#cap.release()
#cv2.destroyAllWindows()

160
160.0
302
302.0
67
67.0
129
129.0
402
402.0
302
302.0


'if (video.isOpened() == False):\n    print("Error en la apertura del fichero {}".format(videos_path + name_video))\n\n# Recuperamos as propieades do video.\nframe_w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))\nframe_h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))\n\nframe_count = 0\nret, prev_frame = video.read()\nwhile(True):\n    ret, current_frame = video.read()\n    if(not ret):\n        break\n\n    # Obtenemos el mapa de saliencia para el frame\n    #gp_sm_frame = PFTVideo_Color_Multiescala(prev_frame, current_frame)  # Fase global\n    pd_sm_frame = PDZhou(prev_frame, current_frame)  # Discrepancia de la fase\n\n    # Leer las fijaciones del frame\n    fix = fixation_maps[\'EyeTrackVDB\'][ivideo][\'Frame\'][0][\'FixL\'][0,frame_count][:,0:2]\n    randNum = np.random.randint(len(fixation_maps[\'EyeTrackVDB\'][ivideo][\'Frame\'][0][\'FixL\'][0]))\n    print(randNum)\n    other_fix = fixation_maps[\'EyeTrackVDB\'][ivideo][\'Frame\'][0][\'FixL\'][0,randNum][:,0:2]\n\n    # Construir e